In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v3.csv", sep = ";", encoding = "utf8", error_bad_lines=False)
#adatbázis első beolvasása

In [ ]:
df.shape
#adatbázis szerkezete (sor és oszlop)

In [ ]:
df.head()
#adatbázis fejléc

In [ ]:
df_new = df[df["onlyday"] >= "2020-03-11"]
#időszakra szűkítés, 2020.03.11. utáni adatok megtartása

In [ ]:
df_new = df_new.reset_index(drop=True)
#újra indexelés

In [ ]:
df_new.shape
#új adatbázis szerkezet

In [ ]:
dictionary = {}
for i in range(len(df_new["id"])):
    dictionary[df_new["id"][i]] = df_new["post_content"][i]
print(list(dictionary.items())[0])
#szótár készítése id - szöveg formában

In [ ]:
df_new.columns
#adatbázis oszlopai

In [ ]:
df_new["post_content"][20]

In [ ]:
df_new["post_content"] = df_new["post_content"].str.replace("â€™", "'")
# â€™ karakterek cseréje '-re, összevonások későbbi kezelése miatt fontos

In [ ]:
df_new["post_content"][20]
#ellenőrzés

In [ ]:
dictionary2 = {}
for key, value in dictionary.items():
    dictionary2.setdefault(value, set()).add(key)

dupl = [list(values) for key, values in dictionary2.items() if len(values) > 1] 
for i in range(len(dupl)):
    del dupl[i][-1]
#szótár készítése duplikációk eltávolítására

In [ ]:
len(dupl)
# hány darab duplikáció van

In [ ]:
print(dupl)

In [ ]:
remove = []
for i in range(len(dupl)):
    for j in range(len(dupl[i])):
        remove.append(dupl[i][j])
#duplikációk törlése

In [ ]:
df_new2 = df_new[~df_new["id"].isin(remove)]
df_new2 = df_new2.reset_index(drop=True)
df_new2.shape
#újra indexelés

In [ ]:
from urlextract import URLExtract
extractor = URLExtract()

In [ ]:
id_txt = []
for file_name in df_new2["post_content"]:
    full = str(file_name)
    full = full.replace('\n', ' ')
    urls = extractor.find_urls(full)
    if len(urls) > 0:
        for u in urls:
            full = full.replace(u, "")
    id_txt.append(full)
#linkek eltávolítása

In [ ]:
len(id_txt)

In [ ]:
df_new2["nolink"]=id_txt

In [ ]:
df_new2.to_csv("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v4.csv", index=False)
df_new2.to_excel("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v4.xlsx", index=False)
#új adatbázis kimentése

In [ ]:
import pandas as pd
df = pd.read_csv("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v4.csv", sep = ",", encoding = "utf8", error_bad_lines=False)
#kimentett adatbázis beolvasása (ne kelljen elölről futtatni)

In [ ]:
txt_hossz = []
for i in range(len(df["nolink"])):
    txt_hossz.append(len(str(df["nolink"][i]).split()))
df['txt_hossz'] = txt_hossz
df_hossz = df[df['txt_hossz'] >= 20]
df_hossz = df_hossz.reset_index(drop = True)
#20 szó alattiak törlése

In [ ]:
df_hossz["post_content"]

In [ ]:
df_hossz.to_excel("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v5.xlsx", index=False)
df_hossz.to_csv("C:/Users/zabor/Desktop/preprocess/depression_data_2020covid_deprred_clean_v5.csv", index=False)
# új adatbázis kimentése

In [ ]:
import itertools
import tqdm
import os
import re
import contractions #ehhez pip install kell

from datetime import datetime

import nltk
nltk.download() #felugró ablak
from nltk.collocations import BigramCollocationFinder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag.stanford import StanfordNERTagger
#csomagok letöltése

In [ ]:
st = StanfordNERTagger('C:/Users/zabor/Desktop/preprocess/adatok/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz',
                       'C:/Users/zabor/Desktop/preprocess/adatok/stanford-ner-2020-11-17/stanford-ner.jar')

lemmatizer = WordNetLemmatizer()

stop_en = stopwords.words('english')
#nevezett entitás felismerés (NER tagger), lemmatizálás, stopszavazás behívása

In [ ]:
person_list = []
person_full = []

def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary=False)

    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #magányos vezetéknevek kigyűjtésének elkerülése
            for part in person:
                name += part + ' '
            person_full.append(name[:-1])
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []
#nevekből lista készítés

In [ ]:
df_hossz.shape

In [ ]:
for i in range(len(df_hossz)):
    f = open("Data/Nolink_txts/"+str(i), "w", encoding='utf-8')
    f.write(str(df_hossz["nolink"][i]))
    f.close()
#szövegek kimentése fájlokba

In [ ]:
import os
#nevek megkeresése a fájlokban
in_path = os.path.join('Data', 'Nolink_txts')
in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]

for file in tqdm.tqdm(in_files):
    try:
        with open(os.path.join(in_path, file), 'r', encoding='utf-8') as article:
            article_text = article.read()
        get_human_names(article_text)
    except Exception as e:
        print(file, e)

#nevek kiírása
ner_file = open(os.path.join('Data', 'ner.tsv'), 'w', encoding='utf-8')
for name in tqdm.tqdm(person_list):
    namelist = name.split()
    line = " ".join(namelist) + "\t" +"_".join(namelist) + "\n"
    line = line.lower()
    ner_file.write(line)
ner_file.close()

In [ ]:
name_list = open(os.path.join('Data', 'ner_nr.tsv'), 'r',
                   encoding='utf-8').readlines()

illness_list = open(os.path.join('Data', 'betegseg.tsv'), 'r',
                    encoding='utf-8').readlines()
#fájlok soronként beolvasása a két változóba

In [ ]:
replace_list = name_list + illness_list


replace_list = open(os.path.join('Data', 'betegseg.tsv'), 'r',
                    encoding='utf-8').readlines()

In [ ]:
replace = []
for r in replace_list:
    w1, w2 = r.strip().split('\t')
    tup = (w1, w2)
    replace.append(tup)

In [ ]:
def filter_names(text, r=replace):
    text = " " + text + " "
    for i in r:
        text = text.replace(" " + i[0] + " ", " " + i[1] + " ")
    return text
#szóközös szavak cseréje alsóvonalas szópárokra

In [ ]:
out_path = os.path.join('Data', 'txts_ner')
folder = [f for f in os.listdir(out_path) if os.path.isfile(os.path.join(out_path, f))]
if len(folder) != 0:
    for f in folder:
        os.remove(os.path.join(out_path, f))
#txts_ner mappából régi fájlok törlése (ha vannak)

In [ ]:
file = 0
for filename in tqdm.tqdm(in_files):
    try:
        article = open(os.path.join(in_path, filename), 'r', encoding='utf-8').read()
        article = article.lower()
        article = re.sub(r'[^\'^\w\s]', '', article)
        names = filter_names(article)
        with open(os.path.join(out_path, in_files[file]), 'w', encoding='utf-8') as out_file:
            out_file.write(names)
    except Exception as e:
        print(in_files[file], e)
    file += 1
#fájlok beolvasása, majd szavak kicserélése betegseg.tsv egyezés alapján és kimentésük a txts_ner mappába

In [ ]:
in_path = os.path.join('Data', 'txts_ner')
in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]

In [ ]:
def fix_contractions(text):
    fixed = contractions.fix(text)
    return fixed
#angol összevonások kezelése pl.: you're -> you are ------> LDA munkafüzetben még kiegészítve

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
#szófajok meghatározása: határozószavak, főnevek, igék, melléknevek

In [ ]:
def stem_filter(text):
    proc_list = []
    doc = text.strip()
    for w in nltk.word_tokenize(doc):
        w = w.lower()
        tok_lemma = lemmatizer.lemmatize(w, get_wordnet_pos(w))
        proc_list.append(tok_lemma)
    return ' '.join(proc_list)
#szavak stemmelése

In [ ]:
out_path_lem = os.path.join('Data', 'txts_stempos')
folder = [f for f in os.listdir(out_path_lem) if os.path.isfile(os.path.join(out_path_lem, f))]
folder_set = set(folder)

In [ ]:
for file in tqdm.tqdm(in_files):
    if file in folder_set:
        continue
    try:
        with open(os.path.join(in_path, file), 'r', encoding='utf-8') as article:
            article_text = article.read()
        fixed_text = fix_contractions(article_text)
        filtered = stem_filter(fixed_text)
        with open(os.path.join(out_path_lem, file), 'w', encoding='utf-8') as outfile:
            outfile.write(filtered)
    except Exception as e:
        print(file, e)
#összevonások kezelése és stemmelés, majd szövegek kimentése

In [ ]:
in_folder = 'txts_stempos'
in_path = os.path.join('Data', in_folder)
in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]

#összes fájl listába rakása
txt_sigbig = []
for filename in tqdm.tqdm(in_files):
    try:
        with open(os.path.join(in_path, filename), 'r', encoding='utf-8') as infile:
            article = infile.read()
        txt_sigbig.append(article)
    except Exception as e:
        print(e)

#szövegekből és szavakból álló listák készítése
txts_sigbig = []
for t in tqdm.tqdm(txt_sigbig):
    txts_sigbig.append(t.strip().split())
words_sigbig = list(itertools.chain(*txts_sigbig))

bigram_measures = nltk.collocations.BigramAssocMeasures()


#függvények definiálása bigramok készítéséhez
def bag_of_words(words):
    return dict([(word, True) for word in words])


#forrás: http://www.nltk.org/howto/collocations.html
def bag_of_bigrams_words(words, score_fn=bigram_measures.pmi, n=500):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigram_finder.apply_freq_filter(min_freq=20)
    bigrams = bigram_finder.nbest(score_fn, n)
    return bag_of_words(bigrams)


#bigramok kimentése fájlba
bigrams_file = open(os.path.join('Data', 'significant_bigrams.tsv'), 'w', encoding='utf-8')

#bigram detektálás
detected_bigrams = bag_of_bigrams_words(words_sigbig)

for bigram in tqdm.tqdm(detected_bigrams):
    line = bigram[0] + ' ' + bigram[1] + "\t" + bigram[0] + "_" + bigram[1] + '\n'
    bigrams_file.write(line)
bigrams_file.close()

#bigramok listájának betöltése
bigram_list = open(os.path.join('Data', 'significant_bigrams_nr.tsv'), 'r',
                   encoding='utf-8').readlines()


#bigramok listába rakása
bigrams = []
for bigram in bigram_list:
    w1, w2 = bigram.strip().split('\t')
    tup = (w1, w2)
    bigrams.append(tup)


#függvény bigramokok cseréjéhez
def filter_bigrams(text, xgrams=bigrams):
    text = " " + text + " "
    for i in xgrams:
        text = text.replace(" " + i[0] + " ", " " + i[1] + " ")
    return text


#fájlok törlése a mappában (ha vannak)
out_path_sigbig = os.path.join('Data', 'txts_bigram')
folder = [f for f in os.listdir(out_path_sigbig) if os.path.isfile(os.path.join(out_path_sigbig, f))]
if len(folder) != 0:
    for f in folder:
        os.remove(os.path.join(out_path_sigbig, f))

#bigramok cseréje
file = 0
for filename in tqdm.tqdm(in_files):
    try:
        article = open(os.path.join(in_path, filename), 'r', encoding='utf-8').read()
        sigbigrams = filter_bigrams(article)
        with open(os.path.join(out_path_sigbig, in_files[file]), 'w', encoding='utf-8') as out_file:
            out_file.write(sigbigrams)
    except Exception as e:
        print(in_files[file], e)
        continue
    file += 1

In [ ]:
in_folder = 'txts_bigram'
in_path = os.path.join('Data', in_folder)
in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]


#szövegek listázása a txts_bigram mappából
txt = []
for filename in tqdm.tqdm(in_files):
    with open(os.path.join(in_path, filename), 'r', encoding='utf-8') as infile:
        article = infile.read()
    txt.append(article)


out_path = os.path.join('Data', 'txts_final')

#stopszavazás
for filename in tqdm.tqdm(in_files):
    try:
        with open(os.path.join(in_path, filename), 'r', encoding='utf-8') as article:
            text = article.read()
        words = text.split()
        article_clean = []
        for word in words:
            if nltk.pos_tag([word])[0][1] == 'PRP' or word not in stop_en:
                article_clean.append(word)
        with open(os.path.join(out_path, filename), 'w', encoding='utf-8') as outfile:
            outfile.write(' '.join(article_clean))
    except Exception as e:
        print(filename, e)

In [ ]:
df["domain2"].value_counts()
#bejegyzések száma források szerint